## windows changing to count FLB
We need two types of FLB to check.
- Fixed FLB for each game
- timely FLB for each game

### timely


In [3]:
%matplotlib inline
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from scipy.interpolate import Rbf
folder_path = '../../datasets/rawdata'
prob_folder_path = '../../datasets/probability'
FLB_folder_path = '../../datasets/FLB'
resam_folder_path = '../../datasets/resample_by_min'
games = { 'baseball':'/baseball/mlb','basketball':'/basketball/nba','football':'/football/nfl',
            'iceball': '/iceball/nhl','soccer_champion':'/soccer/champ_league','scoccer_england':'/soccer/epl',
            'soccer_major':'/soccer/majorleague'}
# folder name is games, result name is games_results.csv

games_list = ['baseball','iceball',
              'soccer_champion','scoccer_england','soccer_major',
              'soccer',
              'soccer_home','soccer_away','soccer_draw']

In [4]:
def generate_percentage_winners(count_df,windows_weight,left,right):
    perc_winnners_obv = pd.DataFrame(columns = ['price','perc_winner'])
    medians = np.array(range(101))/100
    
    for median in medians:
        window_low = median - windows_weight
        window_high = median + windows_weight
        obv_in_windows = count_df[(count_df.iloc[:,0]<=window_high) & (count_df.iloc[:,0]>=window_low)]
        win_obv_in_windows = obv_in_windows[obv_in_windows['HOME_WIN']==True]
        if len(obv_in_windows) < 1:
            continue
        percentage_ = round(len(win_obv_in_windows)/len(obv_in_windows),3)
#         perc_winnners_obv[median] = percentage_
        perc_winnners_obv = perc_winnners_obv.append({'price':median,'perc_winner':percentage_},ignore_index =True)
    perc_winnners_obv = perc_winnners_obv.loc[left:right]
    perc_winnners_obv = perc_winnners_obv.append({'price':0.0,'perc_winner':0.0},ignore_index =True)
    perc_winnners_obv = perc_winnners_obv.append({'price':1.0,'perc_winner':1.0},ignore_index =True)
    perc_winnners_obv = perc_winnners_obv.sort_values(by=['price']).reset_index(drop=True)
    return perc_winnners_obv